In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split

In [2]:
# read sobazaar data
data = pd.read_csv('data/Sobazaar-bprh-cleaned.csv')


In [11]:
original_item_list = sorted(set(data.ItemID))
original_user_list = sorted(set(data.UserID))

data_without_target = data[data.Action != 'P']
data_only_with_target = data[data.Action == 'P']

kf = KFold(n_splits=5, shuffle=True, random_state=20200714)
kf.get_n_splits(data_only_with_target)

5

In [12]:
kf_avg_performance = []
for train_index, test_index in kf.split(data_without_target):
    train_data, test_data = data_without_target.iloc[train_index], data_without_target.iloc[test_index]
    train_data.reset_index(inplace=True, drop=True)
    test_data.reset_index(inplace=True, drop=True)

    # then we construct popularity-based ranking list for item
    ranking_list = train_data.groupby("ItemID").count().UserID.copy()
    ranking_list.sort_values(inplace=True,ascending=False)
    ranking_list = ranking_list.index.to_list()

    # then we get performance metric
    ranking_5 = set(ranking_list[:5])
    ranking_10 = set(ranking_list[:10])
    scoring_list = []
    for u in set(test_data.UserID):
        I_u_t_test = set(test_data[test_data.UserID == u].ItemID)
        p_5 = len(ranking_5.intersection(I_u_t_test)) / 5
        p_10 = len(ranking_10.intersection(I_u_t_test)) / 10
        r_5 = len(ranking_5.intersection(I_u_t_test)) / len(I_u_t_test)
        r_10 = len(ranking_10.intersection(I_u_t_test)) / len(I_u_t_test)
        scoring_list.append([p_5, p_10, r_5, r_10])

    temp = np.array(scoring_list).mean(axis=0)
    kf_avg_performance.append(temp.tolist())

In [13]:
np.array(kf_avg_performance).mean(axis=0)

array([0.03334747, 0.02891174, 0.01978322, 0.03207143])